# HERA Bioinformatics training



In [ ]:
!sed -i -e '1,2d' ~/.bashrc && source ~/.bashrc && bash -c "$(curl -sL https://raw.githubusercontent.com/RIVM-bioinformatics/HERA-Bioinformatics-Training/main/setup.sh)"